<a href="https://colab.research.google.com/github/francescogrillea/AnomalyDetectionTFLite/blob/main/AnomalyDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cat ./drive/MyDrive/Colab*/anomaly_detection_data/res*

In [ ]:
!rm ./drive/MyDrive/Colab*/anomaly_detection_data/res*

# Import e Costanti

In [3]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
import time
import logging

cpu = os.popen('lscpu |grep \'Model name\'').read()
ram_free = os.popen('free -h --si | awk  \'/Mem:/{print $4}\'').read()
ram_total = os.popen('free -h --si | awk  \'/Mem:/{print $2}\'').read()

logging.basicConfig(filename="./drive/MyDrive/Colab Notebooks/anomaly_detection_data/results.log", level=logging.INFO, format='%(message)s')
logging.info('CPU\n\t{}'.format(cpu))
logging.info('RAM\n\t{} / {} free'.format(ram_free[:-1], ram_total[:-1]))

In [82]:
#TODO read yaml file

PROJECT_NAME = 'anomaly_detection'
PROJECT_PATH = './drive/MyDrive/Colab Notebooks/anomaly_detection_data/'
DATASET_PATH = PROJECT_PATH+'Dataset/'
ARCHITECTURE_TYPE = 'LSTM_1L'
TF_MODELS_PATH = PROJECT_PATH+'TF_Models/'+ARCHITECTURE_TYPE+'/'
TFLite_MODELS_PATH = PROJECT_PATH+'TFLite_Models/'+ARCHITECTURE_TYPE+'/'
CHANNEL_NAME = 'P-1'


logging.info('---------------------------------')
logging.info('{}\nArchitecture: {}\nChannel: {}'.format(time.ctime(), ARCHITECTURE_TYPE, CHANNEL_NAME))

#load .h5 model
def load_tf_model(channel_name):
  if not channel_name.endswith('.h5'):
    channel_name = channel_name+'.h5'
  tf_model = keras.models.load_model(TF_MODELS_PATH+channel_name)
  return tf_model

#load dataset
def load_dataset(channel_name):
  if not channel_name.endswith('.npy'):
    channel_name = channel_name+'.npy'
  x = np.array([np.load(DATASET_PATH+'test/'+channel_name)])
  y = np.array([])
  return x,y

# Prediction su TF Model

In [83]:
#load dataset
tf_input_data, tf_output_data = load_dataset(CHANNEL_NAME)
#load model
model = load_tf_model(CHANNEL_NAME)

start_time = time.time()
tf_output_data = model.predict(tf_input_data)
delta_time = time.time()-start_time

logging.info('TensorFlow prediction time: {}s'.format(delta_time))
print('TensorFlow prediction time: {}s'.format(delta_time))

TensorFlow prediction time: 0.7771511077880859s


# Conversione dei modelli da .h5 a .tflite

In [ ]:
channels = [f for f in os.listdir(TF_MODELS_PATH)]

if not CHANNEL_NAME is None:
  channels = [c for c in channels if c.startswith(CHANNEL_NAME)]

for channel in channels:
  tflite_model_path = TFLite_MODELS_PATH+channel[:-3]+'.tflite'

  if not os.path.isfile(tflite_model_path):
    logging.info('Convert {} model from TensorFlow to TensorFlow Lite'.format(CHANNEL_NAME))
    
    start_time = time.time()
    #load TF Model
    tf_model = load_tf_model(channel)
    #STATS- TF model size (in Kb)
    tf_model_size = int(os.path.getsize(TF_MODELS_PATH+channel) / 1024)

    #convert model to TensorFlow Lite
    converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
    converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
  ]
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    #WARNING:absl:Found untraced functions such as lstm_cell -> e' un warning che dipende da come è stato salvato il modello, tuttavia non sembra esserci una precisa soluzione
    tflite_model = converter.convert()

    delta_time = time.time() - start_time

    #generate TFLite model name and path
    tflite_model_filename = channel[:-3]+'.tflite'
    TFLite_MODEL_FILE = TFLite_MODELS_PATH+tflite_model_filename

    #save TFLite model
    with open(TFLite_MODEL_FILE, 'wb') as f:
      f.write(tflite_model)
    #STATS- .tflite model size (in Kb)
    tflite_model_size = int(os.path.getsize(TFLite_MODEL_FILE) / 1024)

    logging.info('Model '+channel[:-3]+' converted (from '+str(tf_model_size)+'Kb to '+str(tflite_model_size)+'Kb) in '+delta_time+'s')
    print('Model '+channel[:-3]+' converted (from '+str(tf_model_size)+'Kb to '+str(tflite_model_size)+'Kb) in '+delta_time+'s')

# Prediction su TFLite

In [84]:
#load dataset
x_data, tflite_output_data = load_dataset(CHANNEL_NAME)

interpreter = tf.lite.Interpreter(model_path=TFLite_MODELS_PATH+CHANNEL_NAME+'.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#preprocessing input data
input_shape = input_details[0]['shape']     #[1, 1, 25]
output_shape = output_details[0]['shape']   #[1, 10]


interpreter.resize_tensor_input(input_details[0]['index'], [1, x_data.shape[1],25])
interpreter.allocate_tensors()

input_data = np.array(x_data, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

start_time = time.time()
#run inference
interpreter.invoke()
tflite_output_data  = interpreter.get_tensor(output_details[0]['index'])
delta_time = time.time() - start_time

logging.info('TensorFlowLite prediction time: {}s'.format(delta_time))
print('TensorFlowLite prediction time: {}s'.format(delta_time))


"""
  Compare TF predictions with TFLite predictions
"""

try:
  for y_tf, y_tflite in zip(tf_output_data[0], tflite_output_data[0]):
    logging.info('{}\t{}\t{}'.format(y_tf, y_tflite, abs(y_tf-y_tflite)))
    print('{}\t{}\t{}'.format(y_tf, y_tflite, abs(y_tf-y_tflite)))

except NameError:
  for val in tflite_output_data[0]:
    print(val)

TensorFlowLite prediction time: 2.0196874141693115s
-0.5264647603034973	-0.5172106027603149	0.009254157543182373
-0.560583770275116	-0.5515011548995972	0.009082615375518799
-0.5594329833984375	-0.5481669902801514	0.011265993118286133
-0.5435227751731873	-0.5302269458770752	0.01329582929611206
-0.5413745045661926	-0.5277824997901917	0.013592004776000977
-0.464438796043396	-0.4500393569469452	0.014399439096450806
-0.4082942008972168	-0.3928990960121155	0.015395104885101318
-0.29708144068717957	-0.2845926880836487	0.012488752603530884
-0.1884109526872635	-0.17934180796146393	0.00906914472579956
-0.18970084190368652	-0.18388013541698456	0.005820706486701965
